## main.py source code

In [1]:
import pymysql
from pymysql.err import OperationalError
from sklearn.neighbors import NearestNeighbors
from flask_restful import reqparse
from flask import jsonify, abort
import numpy  as np
from datetime import datetime

# Constants
radius = 3
n_neighbors = 10
clean_table = 'standard_rentalios'

mysql_config = {
  'user': 'rentalios_admin',
  'password': '9jDHW7czkBGYD',
  'db': 'rentalios',
  'host': '35.225.172.229',
  'charset': 'utf8mb4',
  'cursorclass': pymysql.cursors.DictCursor,
  'autocommit': True
}

# Create SQL connection globally to enable reuse
# PyMySQL does not include support for connection pooling
mysql_conn = None
parser = None

# Helper functions


def change_tofloat(row):
    """
    Helper function for converting rows from a MySQL query into float values.
      Be careful that you know the values can be casted to float, in order to
      avoid errors.
    """
    row[1] = float(row[1])
    row[2] = float(row[2])
    row[0] = abs((row[0] - datetime.now()).days)

    return row


def get_parser():
    """
    Helper function to get build a parser object that can help parsing all the
                    query params from the request.
    """
    parser = reqparse.RequestParser()
    parser.add_argument('latitude', type=float, required=True)
    parser.add_argument('longitude', type=float, required=True)
    parser.add_argument('price', type=int)
    parser.add_argument('sqft', type=int)
    parser.add_argument('private_room', type=int)
    parser.add_argument('private_bath', type=int)
    parser.add_argument('house_type', type=int)
    parser.add_argument('laundry', type=int)
    parser.add_argument('parking', type=int)
    parser.add_argument('cats', type=int)
    parser.add_argument('dogs', type=int)
    parser.add_argument('furnished', type=int)
    parser.add_argument('smoking', type=int)
    parser.add_argument('wheelchair', type=int)

    return parser


def get_cursor():
    """
    Helper function to get a cursor
      PyMySQL does NOT automatically reconnect,
      so we must reconnect explicitly using ping()
    """
    try:
        return mysql_conn.cursor()
    except OperationalError:
        mysql_conn.ping(reconnect=True)
        return mysql_conn.cursor()


def transform_listings(listings):
    """
    Method to extract the necessary information from the listings
    """
    return [i[:12] for i in listings]

# Functions necessary for obtaining the comparables.


def parse_query(request):
    """
    The values from the request are extracted and
    mapped to the user_query which is a list within a list
    """
    global parser

    # Lazy initialization
    if not parser:
        parser = get_parser()

    args = parser.parse_args()

    now = 0
    price = args['price']
    latitude = args['latitude']
    longitude = args['longitude']

    # Check if the required vars are present, else abort.
    if not longitude or not latitude:
        return abort(500)

    # Define user query to be used by the algorithm.
    user_query = [
        now, # Simulates the latest time difference
        latitude,
        longitude,
        args.get('private_room', 0),
        args.get('private_bath', 0),
        args.get('house_type', 0),
        args.get('laundry', 0),
        args.get('parking', 0),
        args.get('furnished', 0),
        args.get('smoking', 0),
        args.get('wheelchair', 0),
        args.get('dogs', 0),  # TODO: Change to 'pet_friendly'
        price,
        args.get('sqft', 0)
    ]

    # Replace all null values with 0
    user_query = [0 if val is None else val for val in user_query]

    print(user_query)

    return [user_query]


def extract_listings(cursor, latitude, longitude):
    """
    Only the listings within a radius of the location given
    by the client are retrieved from the database and mapped
    for the correct type value.
    """
    global radius, clean_table
    query =       """
        SELECT DISTINCT
            run_id, latitude, longitude, private_room, private_bath, house_type, laundry, parking, furnished,
            smoking, wheelchair, pet_friendly, href, title, price, sqft,
            ( 3959
				* acos( cos( radians({lat}) )
						* cos(  radians( latitude ) )
						* cos(  radians( longitude ) - radians({lon}) )
					+ sin( radians({lat}) ) * sin( radians( latitude ) )
					)
			) AS distance
        FROM {table}
        HAVING distance < {radious}
        ORDER BY distance;
		""".format(lat=latitude, lon=longitude,
                   radious=radius, table=clean_table)
    cursor.execute(query)
    print (query)
    results = cursor.fetchall()
    results = [list(i.values()) for i in results]

    return list(map(change_tofloat, results))


def train(listings, user_query):
    global n_neighbors

    # constant
    columns = ['days_old', 'latitude', 'longitude', 'room', 'bath',
                'type', 'laundry', 'parking', 'furnished', 'smoking',
                'wheelchair', 'pet_friendly', 'href', 'title', 'price', 'sqft']

    # If not listings return an empty dictionary
    if listings is []:
        return {}

    n_samples = len(listings)

    # If the number of samples is lower than the number
    # of neighbors then we return only those neighbors
    if n_samples <= n_neighbors:
        results = {}
        for i, listing in enumerate(listings):
            results[i] = dict(zip(columns, listing))
        return results

    # The listings are transformed for the training
    # The user query is transformed for the training
    train_listings = transform_listings(listings)
    train_query = [user_query[0][0:12]]

    # We train the KNN model and return the 10
    # nearest results
    my_pipeline = NearestNeighbors(
        n_neighbors=n_neighbors, algorithm='ball_tree')
    my_pipeline.fit(train_listings)
    _, indexes = my_pipeline.kneighbors(train_query)

    # Mapping of the resulting indexes with their
    # categories
    results = {}
    for i, val in enumerate(indexes[0]):
        results[i] = dict(zip(columns, listings[val]))

    return results


def statistical_analysis(listings):
    # Statistical analysis of the listings retrieved

    if listings is []:
        return 0, 0, 0, {'p15':0, 'p85': 0}

    prices = [i[-3] for i in listings]
    median = int(np.median(prices))
    std = np.std(prices)
    maximum = int(median + std)
    minimum = int(median - std)
    p15, p85 = np.percentile(prices, [15, 85])
    percentile = {'p15': int(p15), 'p85': int(p85)}
    return median, maximum, minimum, percentile


# Main function to execute
def get_comparables(request):
    global mysql_conn

    # Lazy initialization. Doing so minimizes the number of active SQL connections,
    # which helps keep your GCF instances under SQL connection limits.
    if not mysql_conn:
        try:
            mysql_conn = pymysql.connect(**mysql_config)
        except OperationalError:
            # If production settings fail, use local development ones
            mysql_config['unix_socket'] = f'/cloudsql/{CONNECTION_NAME}'
            mysql_conn = pymysql.connect(**mysql_config)

    # Remember to close SQL resources declared while running this function.
    # Keep any declared in global scope (e.g. mysql_conn) for later reuse.
    with get_cursor() as cursor:
        # Get the query from the request.
        user_query = parse_query(request)

        # Take user's query and extract the listings.
        listings = extract_listings(
            cursor, user_query[0][1], user_query[0][2])

        median, maximum, minimum, percentile = statistical_analysis(listings)

        # Then with those listings make a prediction.
        comparables = train(listings, user_query)

        # Create and return JSON object
        return {
            'prediction': {
                'price': median,
                'minimum': minimum,
                'maximum': maximum
            },
            'percentile': percentile,
            'comparables': comparables
               }, 200


def main(request):
        # Set CORS headers for the preflight request
    if request.method == 'OPTIONS':
        # Allows GET requests from any origin with the Content-Type
        # header and caches preflight response for an 3600s
        headers = {
            'Access-Control-Allow-Origin': '*',
            'Access-Control-Allow-Methods': 'GET',
            'Access-Control-Allow-Headers': 'Content-Type',
            'Access-Control-Max-Age': '3600'
        }

        return ('', 204, headers)

    elif request.method == 'GET':
        output = get_comparables(request)

        # Set CORS headers for the main request
        headers = {
            'Access-Control-Allow-Origin': '*'
            # 'Access-Control-Allow-Origin': 'https://app.rentalios.com',
        }

        return (jsonify(output), 200, headers)

    # We only handle OPTIONS and GET methods. For any other, abort.
    else:
        return abort(405)


## Get all listings within radius

In [74]:
# In function extract_listings
global mysql_conn

    # Lazy initialization. Doing so minimizes the number of active SQL connections,
    # which helps keep your GCF instances under SQL connection limits.
if not mysql_conn:
    try:
        mysql_conn = pymysql.connect(**mysql_config)
    except OperationalError:
        # If production settings fail, use local development ones
        mysql_config['unix_socket'] = f'/cloudsql/{CONNECTION_NAME}'
        mysql_conn = pymysql.connect(**mysql_config)
cursor = get_cursor()
#listings = extract_listings(cursor, 37.4636192, -122.144822)
listings = extract_listings(cursor, 37.5962803, -122.0655899)
#listings = extract_listings(cursor, 37.7890183, -122.3915063)


        SELECT DISTINCT
            run_id, latitude, longitude, private_room, private_bath, house_type, laundry, parking, furnished,
            smoking, wheelchair, pet_friendly, href, title, price, sqft,
            ( 3959
				* acos( cos( radians(37.5962803) )
						* cos(  radians( latitude ) )
						* cos(  radians( longitude ) - radians(-122.0655899) )
					+ sin( radians(37.5962803) ) * sin( radians( latitude ) )
					)
			) AS distance
        FROM standard_rentalios
        HAVING distance < 3
        ORDER BY distance;
		


## Transform all listings into standard format

In [75]:
# in function transform_listings
train_listings = transform_listings(listings)
train_listings
#listings
# user_query is the reference point

[[57, 37.597685, -122.062225, 1, 0, 1, 1, 2, 0, 0, 0, 0],
 [44, 37.597685, -122.062225, 1, 0, 1, 1, 2, 0, 0, 0, 0],
 [24, 37.597685, -122.062225, 1, 0, 1, 1, 2, 0, 0, 0, 0],
 [36, 37.593769, -122.062459, 1, 0, 0, 3, 0, 0, 1, 0, 0],
 [34, 37.593288, -122.068834, 1, 0, 1, 3, 0, 1, 1, 0, 0],
 [40, 37.592534, -122.067873, 1, 0, 1, 0, 2, 0, 0, 0, 0],
 [36, 37.592534, -122.067873, 1, 1, 1, 3, 0, 0, 1, 0, 0],
 [28, 37.591791, -122.067118, 1, 0, 1, 3, 0, 0, 1, 0, 0],
 [39, 37.591315, -122.069607, 1, 0, 1, 3, 0, 0, 1, 0, 0],
 [24, 37.591179, -122.061453, 1, 0, 1, 3, 0, 0, 1, 0, 0],
 [21, 37.591024, -122.061089, 1, 0, 1, 3, 0, 0, 1, 0, 0],
 [72, 37.589534, -122.069773, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [37, 37.589534, -122.069773, 1, 0, 0, 1, 1, 0, 1, 0, 0],
 [37, 37.589534, -122.069773, 1, 1, 0, 1, 0, 0, 1, 0, 0],
 [25, 37.589534, -122.069773, 1, 0, 0, 1, 2, 0, 1, 0, 0],
 [57, 37.603137, -122.070362, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [25, 37.603305, -122.071283, 1, 1, 1, 0, 0, 0, 1, 0, 0],
 [72, 37.58790

## Simulate user_query, transfrom to train_query

In [76]:
# in function parse_query
now = 0
latitude = 37.4636192
longitude = -122.144822
# Define user query to be used by the algorithm.
user_query = [
    now, # Simulates the latest time difference
    latitude,
    longitude,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None,
    None, 
    None, # Price: Not used
    None  # SQFT: Not used
]

# Replace all null values with 0
user_query = [0 if val is None else val for val in user_query]

print(user_query)
temp = [user_query]

# in function train
train_query = [temp[0][0:12]] # We are not using price / sqft for KNN prediction
print (train_query)

[0, 37.4636192, -122.144822, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[[0, 37.4636192, -122.144822, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


## Pass into KNN

In [77]:
# in function train
# Fit train_listings in KNN, then pass in reference point train_query
my_pipeline = NearestNeighbors(
n_neighbors=n_neighbors, algorithm='ball_tree')
my_pipeline.fit(train_listings)
_, indexes = my_pipeline.kneighbors(train_query)

## Extract K listings with index returned from KNN

In [78]:
# in function train
results = {}
# constant
columns = ['days_old', 'latitude', 'longitude', 'room', 'bath',
           'type', 'laundry', 'parking', 'furnished', 'smoking',
            'wheelchair', 'pet_friendly', 'href', 'title', 'price', 'sqft']
for i, val in enumerate(indexes[0]):
    results[i] = dict(zip(columns, listings[val]))

In [79]:
prices = []
for i, val in enumerate(indexes[0]):
    prices.append(listings[val][-3])

print (prices)

#prices = [i[-3] for i in listings]
median = int(np.median(prices))
std = np.std(prices)
maximum = int(median + std)
minimum = int(median - std)
p15, p85 = np.percentile(prices, [15, 85])
print ("Min: {}, Med: {}, Max: {}, p15: {}, p85: {}".format(minimum, median, maximum, p15, p85))



[725, 800, 1000, 900, 875, 799, 900, 875, 750, 650]
Min: 739, Med: 837, Max: 934, p15: 733.75, p85: 900.0
